# Iterative prompt development
Usual ML iterative consists of :
> Idea -> Implementation(code/data) -> Experimental results -> Error Analysis -> Repeat

Similarly for Prompt Engineering:
> Try something -> Analyze where the result doesnot give what you want -> Clarify instructions, give more time to think -> Refine Prompts with a batch of examples -> Repeat

In [1]:
import os
import json
import openai
import sys
sys.path.append('../')

# Getting credentials
from Config import openaiConfig
openai.organization = openaiConfig.OPENAI_ORGANISATION
openai.api_key = openaiConfig.OPENAI_API_KEY

In [11]:
def getCompletion(prompt, model='gpt-3.5-turbo'):
    messages = [
    {'role':'user', 'content':prompt}
    ]
    response = openai.ChatCompletion.create(model = model,
                                            messages = messages,
                                            temperature = 0)
    return response.choices[0].message['content']

## Generate a marketing product description from a product fact sheet

In [8]:
factSheetChair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture, 
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) 
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, 
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities: 
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

In [12]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Technical specifications: ```{factSheetChair}```
"""
response = getCompletion(prompt)
print(response)

Introducing our stunning mid-century inspired office chair, the perfect addition to any home or business setting. Part of a beautiful family of office furniture, including filing cabinets, desks, bookcases, meeting tables, and more, this chair is available in several options of shell color and base finishes to suit your style. Choose from plastic back and front upholstery (SWC-100) or full upholstery (SWC-110) in 10 fabric and 6 leather options.

The chair is constructed with a 5-wheel plastic coated aluminum base and features a pneumatic chair adjust for easy raise/lower action. It is available with or without armrests and is qualified for contract use. The base finish options are stainless steel, matte black, gloss white, or chrome.

Measuring at a width of 53 cm, depth of 51 cm, and height of 80 cm, with a seat height of 44 cm and seat depth of 41 cm, this chair is designed for ultimate comfort. You can also choose between soft or hard-floor caster options and two choices of seat fo

### Issue 1 : The length of the response is too long.
limit the length of the response wrt words/ characters

In [17]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Use atmost 50 words.

Technical specifications: ```{factSheetChair}```
"""
response = getCompletion(prompt)
print(response)

Introducing our mid-century inspired office chair, perfect for home or business settings. Available in a range of shell colors and base finishes, with or without armrests. Choose from 10 fabric and 6 leather options for full or plastic upholstery. With a 5-wheel base and pneumatic chair adjust, it's both stylish and functional. Made in Italy.


In [18]:
print(len(response.split(" ")))

55


You can also limit the response with respect to characters

### Issue 2: Focuses on the wrong details
Ask it to focus on the respective details

In [21]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

Use at most 50 words.

At the end of the description, include every 7-character Product ID in the technical specification

Technical specifications: ```{facSheetChair}```
"""
response = getCompletion(prompt)
print(response)

Introducing our mid-century inspired office chair, perfect for home or business settings. With a range of shell colors and base finishes, and the option of plastic or full upholstery, this chair is both stylish and comfortable. Constructed with a 5-wheel plastic coated aluminum base and pneumatic chair adjust, it's also practical. Available with or without armrests and suitable for contract use. Product ID: SWC-100, SWC-110.


## Generate response as HTML data

In [24]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

Use atmost 50 words.

At the end of the description, include every 7-character 
Product ID in the technical specification.

After the description, include a table that gives the 
product's dimensions. The table should have two columns.
In the first column include the name of the dimension. 
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website. 
Place the description in a <div> element.

Technical specifications: ```{factSheetChair}```
"""

response = getCompletion(prompt)
print(response)

<div>
<p>Introducing our mid-century inspired office chair, part of a beautiful family of furniture. Available in multiple shell colors and base finishes, with plastic or full upholstery options. Suitable for home or business settings and qualified for contract use. Constructed with a 5-wheel plastic coated aluminum base and pneumatic chair adjust for easy raise/lower action.</p>
<p>Product IDs: SWC-100, SWC-110</p>
</div>

<table>
  <caption>Product Dimensions</caption>
  <tr>
    <th>Dimension</th>
    <th>Measurement (inches)</th>
  </tr>
  <tr>
    <td>Width</td>
    <td>20.87</td>
  </tr>
  <tr>
    <td>Depth</td>
    <td>20.08</td>
  </tr>
  <tr>
    <td>Height</td>
    <td>31.50</td>
  </tr>
  <tr>
    <td>Seat Height</td>
    <td>17.32</td>
  </tr>
  <tr>
    <td>Seat Depth</td>
    <td>16.14</td>
  </tr>
</table>


### Validating HTML response

In [25]:
from IPython.display import display, HTML

In [26]:
display(HTML(response))

Dimension,Measurement (inches)
Width,20.87
Depth,20.08
Height,31.50
Seat Height,17.32
Seat Depth,16.14
